### import

In [2]:
import torch
from torch import nn
from torch.functional import F
from torch.distributions import Multinomial, Categorical
from torch import optim
from torch.optim import Adam
import numpy as np
import importlib as imp
import actor
import envs
import utils
imp.reload(actor)
imp.reload(envs)
imp.reload(utils)
from actor import Actor, Critic, LinearCritic
from envs import HospitalEnv
from utils import mlp_sample, ppo_update
from env_config import ENV_CONFIG as config
from tqdm import tqdm

In [33]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
env = HospitalEnv(config)
act = Actor(config)
crtc = LinearCritic(config)
optimizer_actor = Adam(act.parameters(), lr=1e-4)
optimizer_critic = Adam(crtc.parameters(), lr=1e-3)

In [5]:
config["Simulation_days"] = 1000
epochs = 5

In [6]:
trajectory = mlp_sample(env, act, config, is_random=True)
states = torch.stack([t['state'] for t in trajectory]).float()
actions = torch.stack([t['action'] for t in trajectory]).float()
costs = torch.tensor([t['cost'] for t in trajectory], dtype=torch.float32)
next_states = torch.stack([t['next_state'] for t in trajectory]).float()

# Recompute log_probs using current logits for compatibility with PPO update
logits = torch.stack([t['logits'] for t in trajectory]).float()
old_probs = F.log_softmax(logits, dim=-1)

memory = (states, actions, old_probs, costs, next_states)
ppo_update(act, crtc, memory, optimizer_actor, optimizer_critic, config)

tensor([ 35.,  35., 175.,  70.])